In [ ]:
import logging
logging.basicConfig(level='ERROR')

In [ ]:
REPEAT_CONSISTENCY = 5

In [ ]:
import pandas as pd

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U ctransformers[cuda]
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 958.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:

In [ ]:
!gdown 1Kr5nfwQ1hdg4VkKPJNmljnc4kdqTyo83 -O SHROOM_test-unlabeled.zip

Downloading...
From: https://drive.google.com/uc?id=1Kr5nfwQ1hdg4VkKPJNmljnc4kdqTyo83
To: /content/SHROOM_test-unlabeled.zip
100% 312k/312k [00:00<00:00, 120MB/s]


In [ ]:
!unzip /content/SHROOM_test-unlabeled.zip

Archive:  /content/SHROOM_test-unlabeled.zip
   creating: SHROOM_test-unlabeled/
  inflating: SHROOM_test-unlabeled/test.model-agnostic.json  
  inflating: SHROOM_test-unlabeled/test.model-aware.json  


01 : Running With LangChain

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=4000,
        do_sample=True,
        top_k=max(REPEAT_CONSISTENCY+1, 5),
        num_return_sequences=REPEAT_CONSISTENCY,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True,
)

In [ ]:
template_mt = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""
template_pg = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""
template_dm = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""

In [ ]:
def chat_with_llm(question , template):
    prompt = template.format(question=question)
    response = pipeline(prompt)
    output = [gen['generated_text'].split('[/INST] </s>')[1].strip() for gen in response]
    return output

In [ ]:
run_on_test = True # whether this baseline system is ran on the test splits or the val splits

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import regex

In [ ]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_aware = "/content/SHROOM_test-unlabeled/test.model-aware.json"
path_val_model_aware_output = "/content/drive/MyDrive/test.model-aware.json"

#from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# simple JSON loading
with open(path_val_model_aware, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)
#num_sample = 5
output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
for i in tqdm.trange(533, num_sample):
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"{src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"{tgt}"

    message = f"Context: {context}\nSentence: {hyp}\n"
    item_to_json=[]
    idx = 0
    responses = chat_with_llm(message, template_mt)
    for response in responses:
      answer = str(response.strip().lower()).replace("```", "")
      match = pattern.findall(answer)[0]
      print(match)
      print("-"*20)
      # p = json.loads(match)
      # item_to_json.append(p)

    # if run_on_test:
    #   item_to_json.append({"id":id})

    # output_json.append(item_to_json)

    # f = open(path_val_model_aware_output, 'w', encoding='utf-8')
    # json.dump(output_json, f)
    # f.close()
print("done")

1500


  0%|          | 0/967 [00:00<?, ?it/s]

{
"answer": "maybe",
"score": 3,
"description": "the context mentions that some roads have been damaged and railway service has been disrupted in affected areas. it also mentions that the noto airport in ishikawa prefecture is closed. however, it is not specified what caused these disruptions or if the sentence is directly related to them."
}
--------------------
{
"answer": "maybe",
"score": 4, 
"description": "while the sentence mentions some elements from the context, it is not clear whether the sentence and context are directly related. the context only mentions specific incidents, while the sentence seems to be a general statement. therefore, while some parts of the sentence may be supported by the context, the overall sentence is not fully supported."
}
--------------------
{
"answer": "yes", 
"score": 4, 
"description": "the context describes the damage to roads and disruption of railway services in affected areas, as well as the closure of the noto airport in ishikawa prefectur